This is draft code using pyxnat to upload large lesion datasets to XNAT.

The code will set up your XNAT connection and then prompt you to select the folder containing the dataset and specify parts of the file naming convention for automation purposes.

Currently designed to handle nii.gz files (if need to gzip files, check test gzip notebook)

To Do: Compress before upload if not .nii.gz, creation of new subject names if original filenames are not in a subject number format (see commented out section), support for non-NIFTI file upload?

Date, Comments/Edits, Author
08-23-17, Initial draft for upload to github, JH
08-29-17, Added user input for selecting resource folder for upload, JH

In [ ]:
#clear
%reset

In [ ]:
#import libraries

import pyxnat
from pyxnat import Interface

import os
from os import listdir
from os.path import isfile, join

from  Tkinter import *
import Tkinter, Tkconstants, tkFileDialog


In [ ]:
#setup xnat connection

connectSuccess = False

while not connectSuccess:
    xnat = Interface(server='http://xnat.bidmc.harvard.edu:8080', cachedir='/tmp')
    listProjects = xnat.select.projects().get()
    
    if not listProjects:
        
        print 'Connection unsuccessful, try again.'
        
    else:
        
        print 'Connection successful, available projects are: '
        print listProjects
        connectSuccess = True


In [ ]:
#select or create new project

exitProjSelect = False

#keep prompting user for project selection
while not exitProjSelect:
    
    #enter project name
    projectName = raw_input('Select project: ')
    projectObj = xnat.select.project(projectName)
    
    #if project entered doesn't exist, ask if want to create
    if not projectObj.exists():
    
        print 'Project does not exist.'
        createProj = raw_input('Create new project? (y/n/anything else to exit) ')
    
        #create new project
        if createProj == 'y':
            
            projectObj.create()
            exitProjSelect = True
            print 'Created new project.'
        
        #go back to project selection if no
        elif createProj == 'n':

            print 'Try again.\n'
        
        #exit project selection
        else:
            
            exitProjSelect = True
            print 'Exiting selection.'
    
    #project exists
    else:
        
        exitProjSelect = True
        print 'Project exists and selected.'
        



In [ ]:
#select folder with data to upload

#bring up GUI to select directory
print('Select folder containing lesions.')
root = Tk()
root.update()
readPath = tkFileDialog.askdirectory()
print ('Selected path is ' + readPath)
root.destroy()



In [ ]:
#get a list of files in the selected directory
onlyFiles = [f for f in listdir(readPath) if isfile(join(readPath, f))]
numFiles = len(onlyFiles)
print('There are ' + str(numFiles) + ' files in this folder.')

   
#find example of compressed nifti file for user to indicate subject number
for i in range(0,numFiles-1):


    if onlyFiles[i].endswith('.nii.gz'):

        print('Here is an example of a filename in the folder ' + onlyFiles[i])
        rightFile = raw_input('Is this example good? (y/n/anything else to exit) ')

        if rightFile == 'y':

            print '\nExample works, give indices of start and end of subject number in filename.'
            exitSubjNumSelect = False

        elif rightFile == 'n':

            print '\nExample does not work, showing next one.'
            exitSubjNumSelect = True

        else:

            exitSubjNumSelect = True
            exitLoop = True
        
        #prompt user to indicate where subject number is in filename
        while not exitSubjNumSelect:

            firstInd = input('First index: ')
            secondInd = input('Second index: ')

            if secondInd <= firstInd:

                print 'Error, second index must be larger than first. Try again.\n'

            else:

                print('Here is what you selected: ' + onlyFiles[i][firstInd:secondInd])
                subjNumCorrect = raw_input('Is this correct? (y/n/anything else to exit): ')

                if subjNumCorrect == 'y':

                    exitSubjNumSelect = True
                    exitLoop = True

                elif subjNumCorrect == 'n':

                    print 'Try again\n'

                else:

                    exitSubjNumSelect = True
                    exitLoop = True

    if exitLoop:

        exitLoop = False
        break #exit for loop
            


In [2]:
#upload files

# ADD IN SOME FUNCTIONALITY FOR GZIP BEFORE UPLOAD IF NOT ALREADY GZIPPED

# ready2Upload = False
# exitUpload = False

# #settings for upload
# while not ready2Upload and not exitUpload:
    
#     resourceFolder = raw_input('Which resource folder would you like to upload these files? (Lesion/Connectivity/etc) ')

#     newSubj = raw_input('Upload files as new subject number (e.g.filenames do not correpsond to a subject number)? (y/n) ')
    
#     if newSubj == 'y':
        
#         storeFileName = raw_input('Store original filename with upload? (y/n) ')
        
#         if storeFileName == 'y':
            
#             storeFileNameFlag = True
            
#         else:
            
#             storeFileNameFlag = False
        
#         newSubjFlag = True
        
#     else:

#         newSubjFlag = False
        
#     print '\nThese are the upload settings you have indicated:'
#     print('Resource folder for upload: ' + resourceFolder)
#     print('Create new subject number: ' + str(newSubjFlag))
    
#     if newSubjFlag:
        
#         print('Store original filename with upload: ' + str(storeFileNameFlag))
        
#     proceed = raw_input('Are these settings correct and proceed with upload? (y/n/anything else to cancel) ')
    
#     if proceed =='y':
    
#         ready2Upload = True
        
#     elif proceed =='n':
        
#         print 'Input settings again.'
#         ready2Upload = False
        
#     else:
        
#         exitUpload = True
        
#actual upload    
overwriteFlag = False

for fileName in onlyFiles:
    
    print('\nCurrent file is ' + fileName)   
    
#     if newSubjFlag:
        
#         if storeFileNameFlag:
            
            
        
#     else:
        
    print('Subject number is ' + fileName[firstInd:secondInd])
    subjectObj = projectObj.subject(fileName[firstInd:secondInd])
    
    
    if not subjectObj.exists():
        
        print('Subject does not exist, creating new subject folder.')
        subjectObj.create()
        
    if subjectObj.resource(resourceFolder).file(fileName).exists():
        
        if not overwriteFlag:
        
            overwrite = raw_input('File already exists. Overwrite? (y/n/y2all) ')
        
            if overwrite == 'y2all':
            
                overwriteFlag = True
        
        if overwrite == 'y' or overwrite == 'y2all':
            
            subjectObj.resource(resourceFolder).file(fileName).delete()
            print 'Original file deleted, uploading new copy.'
            
        else:
            
            continue
            
            
    projectObj.subject(fileName[firstInd:secondInd]).resource(resourceFolder).file(fileName).insert(readPath + '/' + fileName)
    print('File uploaded.\n')


Which resource folder would you like to upload these files? (Lesion/Connectivity/etc) C
Upload files as new subject number (e.g.filenames do not correpsond to a subject number)? (y/n) y
Store original filename with upload? (y/n) n

These are the upload settings you have indicated:
Resource folder for upload: C
Create new subject number: True
Store original filename with upload: False
Are these settings correct and proceed with upload? (y/n/anything else to cancel) n
Input settings again.
Which resource folder would you like to upload these files? (Lesion/Connectivity/etc) Lesion
Upload files as new subject number (e.g.filenames do not correpsond to a subject number)? (y/n) n

These are the upload settings you have indicated:
Resource folder for upload: Lesion
Create new subject number: False
Are these settings correct and proceed with upload? (y/n/anything else to cancel) sdf


In [ ]:
# delete stuff

# for fileName in onlyFiles[0:226]:

#     subjectObj = projectObj.subject(fileName[7:10])
    
#     print subjectObj
    
#     if subjectObj.resource('Connectivity').file(fileName).exists():
        
#         subjectObj.resource('Connectivity').file(fileName).delete()
#         print 'file exists and deleted\n'
        

# onlyFiles[226]
# for fileName in onlyFiles[0:226]